Workflow
Steps-
1. Import pandas , request and pycountry libraries
    - pandas to create dataframes 
    - requests to work with APIs
    - pycountry to work with geographical data
2. WebScrap data of under 5 mordlity rate 
3. Call an api to get data on growth of the countries 
4. Clean Country Codes with a function that return 
5. Save CSV for SQL

Import Libraries

In [1]:
import pandas as pd 
import requests
import pycountry

Under-5 Mortality

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_countries_by_infant_and_under-five_mortality_rates'
t= pd.read_html(url)
mortality_df =t[0]
mortality_df.head()

,Location,2020 rate
0,Afghanistan *,58.0
1,Albania *,9.8
2,Algeria *,23.7
3,Andorra *,2.5
4,Angola *,71.5


GDP Growth per country

In [3]:
api_url = 'https://api.worldbank.org/v2/country/all/indicator/NY.GDP.MKTP.KD.ZG?date=2020&format=json&per_page=20000'
response = requests.get(api_url)
data = response.json()[1] #data inside 2nd element
gdp_df = pd.DataFrame(data)
gdp_df

,indicator,country,countryiso3code,date,value,unit,obs_status,decimal
0,"{'id': 'NY.GDP.MKTP.KD.ZG', 'value': 'GDP grow...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2020,-2.859784,,,1
1,"{'id': 'NY.GDP.MKTP.KD.ZG', 'value': 'GDP grow...","{'id': 'ZI', 'value': 'Africa Western and Cent...",AFW,2020,-0.984117,,,1
2,"{'id': 'NY.GDP.MKTP.KD.ZG', 'value': 'GDP grow...","{'id': '1A', 'value': 'Arab World'}",ARB,2020,-4.757145,,,1
3,"{'id': 'NY.GDP.MKTP.KD.ZG', 'value': 'GDP grow...","{'id': 'S3', 'value': 'Caribbean small states'}",CSS,2020,-9.113253,,,1
4,"{'id': 'NY.GDP.MKTP.KD.ZG', 'value': 'GDP grow...","{'id': 'B8', 'value': 'Central Europe and the ...",CEB,2020,-3.315368,,,1
...,...,...,...,...,...,...,...,...
261,"{'id': 'NY.GDP.MKTP.KD.ZG', 'value': 'GDP grow...","{'id': 'VI', 'value': 'Virgin Islands (U.S.)'}",VIR,2020,-1.614938,,,1
262,"{'id': 'NY.GDP.MKTP.KD.ZG', 'value': 'GDP grow...","{'id': 'PS', 'value': 'West Bank and Gaza'}",PSE,2020,-11.318466,,,1
263,"{'id': 'NY.GDP.MKTP.KD.ZG', 'value': 'GDP grow...","{'id': 'YE', 'value': 'Yemen, Rep.'}",YEM,2020,NaN,,,1
264,"{'id': 'NY.GDP.MKTP.KD.ZG', 'value': 'GDP grow...","{'id': 'ZM', 'value': 'Zambia'}",ZMB,2020,-2.785055,,,1


Data Cleaning

Check how Both table "country name" looks like 

In [4]:
print(type(mortality_df['Location'].iloc[0]))


<class 'str'>


In [5]:
print(mortality_df['Location'].unique()[:20])

['Afghanistan\u202f*' 'Albania\u202f*' 'Algeria\u202f*' 'Andorra\u202f*'
 'Angola\u202f*' 'Antigua and Barbuda\u202f*' 'Argentina\u202f*'
 'Armenia\u202f*' 'Australia\u202f*' 'Austria\u202f*' 'Azerbaijan\u202f*'
 'Bahamas' 'Bahrain\u202f*' 'Bangladesh\u202f*' 'Barbados\u202f*'
 'Belarus\u202f*' 'Belgium\u202f*' 'Belize\u202f*' 'Benin\u202f*'
 'Bhutan\u202f*']


In [6]:
print(type(gdp_df['country'].iloc[0]))


<class 'dict'>


In [7]:
print(gdp_df['country'].head(20))

0     {'id': 'ZH', 'value': 'Africa Eastern and Sout...
1     {'id': 'ZI', 'value': 'Africa Western and Cent...
2                   {'id': '1A', 'value': 'Arab World'}
3       {'id': 'S3', 'value': 'Caribbean small states'}
4     {'id': 'B8', 'value': 'Central Europe and the ...
5     {'id': 'V2', 'value': 'Early-demographic divid...
6          {'id': 'Z4', 'value': 'East Asia & Pacific'}
7     {'id': '4E', 'value': 'East Asia & Pacific (ex...
8     {'id': 'T4', 'value': 'East Asia & Pacific (ID...
9                    {'id': 'XC', 'value': 'Euro area'}
10       {'id': 'Z7', 'value': 'Europe & Central Asia'}
11    {'id': '7E', 'value': 'Europe & Central Asia (...
12    {'id': 'T7', 'value': 'Europe & Central Asia (...
13              {'id': 'EU', 'value': 'European Union'}
14    {'id': 'F1', 'value': 'Fragile and conflict af...
15    {'id': 'XE', 'value': 'Heavily indebted poor c...
16                 {'id': 'XD', 'value': 'High income'}
17                   {'id': 'XF', 'value': 'IBRD

Clean data

1. in mortality dataframe \u202f* after text will hamper the preparation of tables . we need to replace that
2. we need to create a abbreviation column that call help us to join tables
3. in gdp dataframe to create abbreviation as in dict , id contain shorten we need to extract that

Clean the mortality Location column


In [8]:
def clean_country_name(name):
    # remove \u202f , space , * 
    name = name.replace("\u202f","")
    name = name.replace("*","")
    name = name.strip() #remove extra spaces
    return name 

Prepare the Tables 

Mortality_df new Column abb

In [9]:
def to_abb(name):
    try:
        clean_name = clean_country_name(name)
        return pycountry.countries.lookup(clean_name).alpha_3
    except:
        return None

In [10]:
mortality_df['abb'] = mortality_df['Location'].apply(to_abb)
mortality_df

,Location,2020 rate,abb
0,Afghanistan *,58.0,AFG
1,Albania *,9.8,ALB
2,Algeria *,23.7,DZA
3,Andorra *,2.5,AND
4,Angola *,71.5,AGO
...,...,...,...
188,Vietnam *,20.9,VNM
189,Palestine *,16.5,None
190,Yemen *,59.6,YEM
191,Zambia *,61.4,ZMB


GDP dataframe new column abb

In [11]:
gdp_df['abb'] = gdp_df['country'].apply(lambda x : x['id'])
gdp_df

,indicator,country,countryiso3code,date,value,unit,obs_status,decimal,abb
0,"{'id': 'NY.GDP.MKTP.KD.ZG', 'value': 'GDP grow...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2020,-2.859784,,,1,ZH
1,"{'id': 'NY.GDP.MKTP.KD.ZG', 'value': 'GDP grow...","{'id': 'ZI', 'value': 'Africa Western and Cent...",AFW,2020,-0.984117,,,1,ZI
2,"{'id': 'NY.GDP.MKTP.KD.ZG', 'value': 'GDP grow...","{'id': '1A', 'value': 'Arab World'}",ARB,2020,-4.757145,,,1,1A
3,"{'id': 'NY.GDP.MKTP.KD.ZG', 'value': 'GDP grow...","{'id': 'S3', 'value': 'Caribbean small states'}",CSS,2020,-9.113253,,,1,S3
4,"{'id': 'NY.GDP.MKTP.KD.ZG', 'value': 'GDP grow...","{'id': 'B8', 'value': 'Central Europe and the ...",CEB,2020,-3.315368,,,1,B8
...,...,...,...,...,...,...,...,...,...
261,"{'id': 'NY.GDP.MKTP.KD.ZG', 'value': 'GDP grow...","{'id': 'VI', 'value': 'Virgin Islands (U.S.)'}",VIR,2020,-1.614938,,,1,VI
262,"{'id': 'NY.GDP.MKTP.KD.ZG', 'value': 'GDP grow...","{'id': 'PS', 'value': 'West Bank and Gaza'}",PSE,2020,-11.318466,,,1,PS
263,"{'id': 'NY.GDP.MKTP.KD.ZG', 'value': 'GDP grow...","{'id': 'YE', 'value': 'Yemen, Rep.'}",YEM,2020,NaN,,,1,YE
264,"{'id': 'NY.GDP.MKTP.KD.ZG', 'value': 'GDP grow...","{'id': 'ZM', 'value': 'Zambia'}",ZMB,2020,-2.785055,,,1,ZM


Save to CSV for SQL

In [12]:
mortality_df.to_csv("mortarlity.csv",index=False)
gdp_df.to_csv("gdp.csv",index=False)